In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools as ft
import numpy as np
import pandas as pd
from time import sleep
import re
from pprint import pprint
import time

In [2]:
today_mm, today_dd = list(map(int,time.strftime('%m.%d').split('.')))
today_mm, today_dd

(4, 29)

In [4]:
year_max, year_min = 2020,2000
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages

Range : 2000 ~ 2020
Year 2020 - 19 pages
Year 2019 - 43 pages
Year 2018 - 50 pages
Year 2017 - 53 pages
Year 2016 - 59 pages
Year 2015 - 71 pages
Year 2014 - 76 pages
Year 2013 - 61 pages
Year 2012 - 44 pages
Year 2011 - 41 pages
Year 2010 - 29 pages
Year 2009 - 24 pages
Year 2008 - 28 pages
Year 2007 - 26 pages
Year 2006 - 24 pages
Year 2005 - 21 pages
Year 2004 - 38 pages
Year 2003 - 20 pages
Year 2002 - 18 pages
Year 2001 - 17 pages
Year 2000 - 19 pages


[('2020', '19'),
 ('2019', '43'),
 ('2018', '50'),
 ('2017', '53'),
 ('2016', '59'),
 ('2015', '71'),
 ('2014', '76'),
 ('2013', '61'),
 ('2012', '44'),
 ('2011', '41'),
 ('2010', '29'),
 ('2009', '24'),
 ('2008', '28'),
 ('2007', '26'),
 ('2006', '24'),
 ('2005', '21'),
 ('2004', '38'),
 ('2003', '20'),
 ('2002', '18'),
 ('2001', '17'),
 ('2000', '19')]

In [5]:
csvdata,csvtotal = [], 0
for year, pages in year_pages:
    for page in range(1,int(pages)+1):
        try:
            soup = BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open='+year+'&page='+str(page)),'html.parser')
            collected, total = 0, 0
            for link in soup.find('ul','directory_list').select('li > a'):
                err_msg = ''
                try:
                    date = re.findall('\d{2}.\d{2}',link.text)

                    if 'href' in link.attrs and 'class' not in link.attrs:
                        basic_page = 'https://movie.naver.com'+link.attrs['href']
                        data = BeautifulSoup(urlopen(basic_page),'html.parser')
                        total += 1

                        # title
                        title = data.find('h3','h_movie').select('a')[0].text

                        msg = 'error at date'
                        # date
                        date = data.find('dl','info_spec').find('dd').find_all('span')[-1].find_all('a')[-1].text[1:]
                        
                        msg = 'error at rating & voting'
                        # rating, vote_count
                        rating = str(ft.reduce(lambda x,y:x+y,[val.text for val in data.find('div','score score_left').select('div > a > em')[:4]]))
                        vote_count = data.find('div','score score_left').find('div','ly_count').em.text
                            
                        msg = 'error at genre'
                        # genre
                        genre = '|'.join(list(map(lambda x: x.text, filter(lambda x: 'genre' in x.attrs['href'], data.find('dl','info_spec').select('dd > p > span > a')))))
                        #genre = genre.replace('멜로/로맨스','로맨스')
                        
                        msg = 'error at plot'
                        #plot
                        plot = data.find_all('p','con_tx')[0].text
                        
                        msg = 'error at actors'
                        # actors
                        detail_page = basic_page.replace('basic','detail')
                        detail_soup = BeautifulSoup(urlopen(detail_page))

                        main_act,supp_act = '',''
                        main_num = sum([1 if line.text == '주연' else 0 for line in detail_soup.find_all('em','p_part')])
                        if main_num > 0:
                            for i,actor in enumerate(detail_soup.find('ul','lst_people').find_all('a','k_name')):
                                if i<main_num:
                                    main_act += actor.text+'|'
                                else:
                                    supp_act += actor.text+'|'
                            main_act,supp_act = main_act[:-1],supp_act[:-1]

                        msg = 'error at img'
                        #img_url
                        img_url = data.find('div','poster').find('img').attrs['src']

                        # data check => save
                        if rating != '0.00' and genre != '' and len(date) > 0:
                            '''
                            영화제목, 장르, 연도, 날짜, 평점, 투표수, 주연, 조연, url, 이미지링크
                            '''
                            csvdata.append([title, genre, year, date, rating, vote_count,
                                           plot, main_act, supp_act, basic_page,img_url])
                            collected += 1
                            csvtotal += 1
                        
                except Exception as e:
                    print('Error at title({}) & {}'.format(title,msg))
            print('year {0} page {1}/{2} - crawling {3}/{4} completed - {5}'.format(year,page,pages,collected,total,csvtotal))
            #sleep(3)
        except Exception as e:
            print('year {0} page {1}/{2} - crawling {3}/{4} failed - {5}'.format(year,page,pages,collected,total,csvtotal))
        if page % 10 == 0:
            print('Check per 10 pages - Recently appended 10 data')
            pprint(csvdata[-5:])

Error at title(007 노 타임 투 다이) & error at rating & voting


c:\users\jbose\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\jbose\appdata\local\programs\python\python36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error at title(결백) & error at rating & voting
Error at title(고스트버스터즈 라이즈) & error at rating & voting
Error at title(고양이 집사) & error at rating & voting
Error at title(골든타임) & error at rating & voting
year 2020 page 1/19 - crawling 14/20 completed - 14
Error at title(광견 아토스) & error at rating & voting
Error at title(국제수사) & error at rating & voting
Error at title(그레텔과 헨젤) & error at rating & voting
Error at title(그집) & error at rating & voting
Error at title(극장판 포켓몬스터 뮤츠의 역습 EVOLUTION) & error at rating & voting
Error at title(기도의 막이 내릴 때) & error at rating & voting
Error at title(꼴통 2인조) & error at rating & voting
year 2020 page 2/19 - crawling 12/20 completed - 26
Error at title(나는보리) & error at rating & voting
Error at title(나의 촛불) & error at rating & voting
Error at title(낙원의 밤) & error at rating & voting
Error at title(내가 죽던 날) & error at rating & voting
Error at title(다만 악에서 구하소서) & error at rating & voting
year 2020 page 3/19 - crawling 13/20 completed - 39
Error at title(더 프린세스 :

Error at title(이상한 나라의 수학자) & error at rating & voting
Error at title(이터널 코드) & error at rating & voting
Error at title(인간실격) & error at rating & voting
Error at title(인베이젼 2020) & error at rating & voting
Error at title(인질(가제)) & error at rating & voting
Error at title(저 산 너머) & error at rating & voting
Error at title(정상회담) & error at rating & voting
year 2020 page 14/19 - crawling 12/20 completed - 153
Error at title(제8일의 밤) & error at rating & voting
Error at title(조금씩, 천천히 안녕) & error at rating & voting
Error at title(초미의 관심사) & error at rating & voting
year 2020 page 15/19 - crawling 14/20 completed - 167
Error at title(출장수사) & error at rating & voting
Error at title(침입자) & error at rating & voting
Error at title(카페 벨에포크) & error at rating & voting
Error at title(캣츠토피아) & error at rating & voting
Error at title(콜) & error at rating & voting
Error at title(콜 오브 와일드) & error at rating & voting
Error at title(콰이어트 플레이스 2) & error at rating & voting
Error at title(킬러의 보디가드 무삭제 특별판) & 

year 2019 page 21/43 - crawling 19/20 completed - 583
year 2019 page 22/43 - crawling 20/20 completed - 603
year 2019 page 23/43 - crawling 19/20 completed - 622
year 2019 page 24/43 - crawling 18/20 completed - 640
year 2019 page 25/43 - crawling 20/20 completed - 660
year 2019 page 26/43 - crawling 19/20 completed - 679
year 2019 page 27/43 - crawling 19/20 completed - 698
year 2019 page 28/43 - crawling 18/20 completed - 716
year 2019 page 29/43 - crawling 20/20 completed - 736
year 2019 page 30/43 - crawling 17/20 completed - 753
Check per 10 pages - Recently appended 10 data
[['우키시마호',
  '다큐멘터리',
  '2019',
  '09.19',
  '6.54',
  '121',
  '1945년 8월 22일. \r'
  '\xa0조선인 만여 명을 태운 귀국선, 우키시마호가 출항을 시작한다. \r'
  '\xa0목적지는 부산이었다. \r'
  '\xa0\r'
  '\xa0하지만 출항 이틀 후, \r'
  '\xa0우키시마호는 갑작스럽게 폭발 \r'
  '\xa0그리고 침몰.. \r'
  '\xa0배에 탄 수천 명의 조선인이 목숨을 잃었다. \r'
  '\xa0\r'
  '\xa0사건이 발생한 지 74년, \r'
  '\xa0우키시마호 폭발 원인은 여전히 밝혀지지 않고 있다. \r'
  '\xa0이제 우리가, 그 원인을 밝혀야만 한다.',
  '안재모',
  '',
  'https://movie.na

...

Error at title(신데렐라 스토리) & error at plot
year 2004 page 21/38 - crawling 12/20 completed - 10928
Error at title(썸머랜드) & error at plot
year 2004 page 22/38 - crawling 11/20 completed - 10939
year 2004 page 23/38 - crawling 15/20 completed - 10954
year 2004 page 24/38 - crawling 12/20 completed - 10966
Error at title(엘라 인챈티드) & error at plot
year 2004 page 25/38 - crawling 13/20 completed - 10979
Error at title(올드 미스 다이어리) & error at plot
year 2004 page 26/38 - crawling 12/20 completed - 10991
Error at title(웰컴 프레지던트) & error at plot
Error at title(유니미니펫) & error at plot
year 2004 page 27/38 - crawling 13/20 completed - 11004
year 2004 page 28/38 - crawling 13/20 completed - 11017
Error at title(자등련) & error at plot
year 2004 page 29/38 - crawling 12/20 completed - 11029
year 2004 page 30/38 - crawling 14/20 completed - 11043
Check per 10 pages - Recently appended 10 data
[['착신아리',
  '공포|스릴러',
  '2004',
  '07.09',
  '7.03',
  '785',
  '여대생인 유미(中村由美, 시바사키 코우 分)는 어느 날 친구가 주선한 미팅에 나갔다가 서로 휴

year 2003 page 9/20 - crawling 12/20 completed - 11290
Error at title(스카이 어드벤처) & error at plot
year 2003 page 10/20 - crawling 17/20 completed - 11307
Check per 10 pages - Recently appended 10 data
[['신밧드 - 7대양의 전설',
  '모험|애니메이션|액션|가족',
  '2003',
  '07.11',
  '8.56',
  '186',
  '넘치는 혈기와 모험심으로 광활한 바다를 항해하며 약탈을 일삼는 도적계의 섹시남, 신밧드! 거칠 것 없는 그의 용기와 패기 앞에 엄청난 '
  "사건이 도사리고 있었으니... 그것은 바로 세계의 평화를 좌지우지할 '평화의 책' 도난 사건. 평화의 책이 사라지는 순간, 세상은 "
  "일순간에 어둠에 휩싸이고 사람들은 혼란에 빠지게 된다.\xa0 혼돈의 여신 에리스의 모략으로 '평화의 책'을 훔친 범인으로 지목된 "
  "신밧드는 위기에 몰리게 된다. '평화의 책'을 수호하는 임무를 맡은 그의 절친한 친구 프로테우스는 신밧드를 대신해 감옥에 갇히고, 만약 "
  "신밧드가 '평화의 책'을 찾아오지 못하면 프로테우스는 죽음을 면치 못하게 된다. 그러나 프로테우스의 죽음은 아랑곳 않고 '피지'로 놀러 "
  '갈 계획만 세우는 신밧드. 그 앞에 프로테우스의 약혼녀인 마리나가 등장해 그의 최대의 약점인 돈을 무기로 신밧드를 매수한다.\xa0 '
  "신밧드는 당돌하기 그지없는 마리나에게 이끌려, 결국 '평화의 책'을 찾기 위한 대장정에 오르게 되는데... 그들 앞에 놓여있는 수많은 "
  "공포와 위험. 과연 그들은 혼돈의 여신 에리스에 맞서 '평화의 책'을 되찾고, 프로테우스의 목숨을 구할 수 있을 것인가.",
  '브래드 피트|캐서린 제타 존스',
  '미셸 파이퍼|조셉 파인즈|데니스 헤이스버트|티모시 웨스트|아드리아노 지안니니|라맨 허|짐 커밍스|콘래드 버논|크리스 밀러

Error at title(CSI - 마이애미) & error at plot
year 2002 page 1/18 - crawling 17/20 completed - 11490
Error at title(그 여자 사람잡네) & error at plot
Error at title(그대를 알고부터) & error at plot
Error at title(기동전사 건담 시드) & error at plot
year 2002 page 2/18 - crawling 14/20 completed - 11504
Error at title(꽃보다 남자 2) & error at plot
Error at title(동물원 사람들) & error at plot
year 2002 page 3/18 - crawling 14/20 completed - 11518
Error at title(디지몬 테이머즈) & error at plot
Error at title(라이벌) & error at plot
year 2002 page 4/18 - crawling 14/20 completed - 11532
year 2002 page 5/18 - crawling 18/20 completed - 11550
year 2002 page 6/18 - crawling 16/20 completed - 11566
Error at title(미스터 빈 - 애니메이션 시리즈) & error at plot
Error at title(믿거나 말거나 - 3차 TV 시리즈) & error at plot
Error at title(별을 쏘다) & error at plot
year 2002 page 7/18 - crawling 14/20 completed - 11580
year 2002 page 8/18 - crawling 16/20 completed - 11596
Error at title(선물) & error at plot
Error at title(순수의 시대) & error at plot
year 2002 page 9/18

Error at title(언제나 두근두근) & error at plot
year 2002 page 11/18 - crawling 16/20 completed - 11647
Error at title(여고 동창생) & error at plot
Error at title(여우와 솜사탕) & error at plot
Error at title(오렌지) & error at plot
year 2002 page 12/18 - crawling 12/20 completed - 11659
Error at title(이누야샤) & error at plot
year 2002 page 13/18 - crawling 18/20 completed - 11677
Error at title(잘난 걸 어떡해) & error at plot
Error at title(정) & error at plot
year 2002 page 14/18 - crawling 16/20 completed - 11693
year 2002 page 15/18 - crawling 13/20 completed - 11706
Error at title(킴 파서블) & error at plot
Error at title(파워레인저 와일드 포스) & error at plot
year 2002 page 16/18 - crawling 18/20 completed - 11724
Error at title(하늘에서 내리는 일억개의 별) & error at plot
year 2002 page 17/18 - crawling 18/20 completed - 11742
year 2002 page 18/18 - crawling 17/18 completed - 11759
Error at title(TV 동물농장) & error at plot
Error at title(UC - 언더커버) & error at plot
year 2001 page 1/17 - crawling 16/20 completed - 11775
Error at title(그

Error at title(논스톱 2 - 뉴 논스톱) & error at plot
Error at title(덕이) & error at plot
year 2000 page 4/19 - crawling 17/20 completed - 12099
Error at title(디지몬 어드벤처) & error at plot
Error at title(러브 히나) & error at plot
Error at title(레스트리스) & error at img
Error at title(로드 트립) & error at img
year 2000 page 5/19 - crawling 12/20 completed - 12111
Error at title(멋진 친구들) & error at plot
year 2000 page 6/19 - crawling 15/20 completed - 12126
year 2000 page 7/19 - crawling 18/20 completed - 12144
Error at title(비밀) & error at plot
Error at title(사랑의 전설) & error at plot
year 2000 page 8/19 - crawling 16/20 completed - 12160
Error at title(성난 얼굴로 돌아보라) & error at plot
Error at title(세크릿 에이전트 맨) & error at img
year 2000 page 9/19 - crawling 17/20 completed - 12177
Error at title(슬리피 할로우) & error at img
year 2000 page 10/19 - crawling 17/20 completed - 12194
Check per 10 pages - Recently appended 10 data
[['슬리피 할로우',
  '판타지|스릴러|액션|공포',
  '2000',
  '01.29',
  '8.17',
  '642',
  "1799년 뉴욕의 북쪽에 위치한 '슬

In [6]:
len(csvdata)

12330

In [31]:
df = pd.DataFrame(csvdata,columns=['title','genre','year','date','rating','vote_count','plot', 'main_act','supp_act','page_url','img_url'])
df.head()

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,02.19,8.88,"5,017",제1차 세계대전이 한창인 1917년.\r 독일군에 의해 모든 통신망이 파괴된 상황 ...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,01.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,04.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,02.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,,,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,03.02,10.0,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


In [32]:
data = df['plot'][3]
display(data)
display(re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(data).strip())).strip())

'곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 커플 여행을 제안한다. \r\xa0지연은 불편한 여행이 마음에 들지 않지만 용우의 직장 상사라 어쩔 수 없이 승낙하게 된다. \r\xa0함께 여행을 가기로 확정이 되고 차욱은 평소 눈여겨 보던 지연과 함께 할 수 있도록 계획을 세운다. \r\xa0민주는 차욱의 강요에 어쩔 수 없이 계획에 동참하게 된다. \r\xa0여행 당일 민주는 결국 지연에게 모든 것을 털어놓고 지연은 차욱을 반대로 이용해 먹기로 한다.'

'곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 커플 여행을 제안한다. 지연은 불편한 여행이 마음에 들지 않지만 용우의 직장 상사라 어쩔 수 없이 승낙하게 된다. 함께 여행을 가기로 확정이 되고 차욱은 평소 눈여겨 보던 지연과 함께 할 수 있도록 계획을 세운다. 민주는 차욱의 강요에 어쩔 수 없이 계획에 동참하게 된다. 여행 당일 민주는 결국 지연에게 모든 것을 털어놓고 지연은 차욱을 반대로 이용해 먹기로 한다.'

In [33]:
df['plot'] = df['plot'].map(lambda x: re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(x).strip())).strip())

In [34]:
df['vote_count'] = df['vote_count'].apply(lambda x: int(x.replace(',','')))

In [35]:
import datetime
today = datetime.datetime.today()
mmdd = str(today.month) + str(today.day)

df.to_csv('./dataset/naver_movie_dataset_'+mmdd+'.csv',encoding='utf-8',index=False)
del df

In [36]:
df = pd.read_csv('./dataset/naver_movie_dataset_429.csv',)
df.head(20)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...
5,갱,액션,2020,1.16,5.99,105,교도소를 개조해 만든 국내 최악의 고등학교 ‘대훈고’ 그리고 그 곳에 온 강제전학생...,차지혁|조선기|옥윤중|백재민|김대한|이정현,김라희|송은석|진성|이재한|맹재렬|이채성|최준민|시윤|지남혁|임정옥|권오윤|김동우|...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191115_141/1...
6,건즈 아킴보,액션,2020,4.15,6.39,173,"낮엔 평범한 직장인, 밤엔 키보드 워리어 ‘마일즈’ 손가락 한번 잘못 놀렸다가 살인...",다니엘 래드클리프|사마라 위빙|네드 데네히|나타샤 류 보르디초,라이스 다비|재클린 리 거츠|콜린 모이,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200323_116/1...
7,걸즈 앤 판처 제 63회 전차도 전국 고교생 대회,애니메이션,2020,4.09,4.67,3,여성의 소양 중 하나로 손꼽히며 전통적인 무예의 교양 활동인 전차도로 유명한 니시즈...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200330_55/15...
8,걸즈 앤 판처 최종장,애니메이션,2020,3.12,8.93,15,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200303_117/1...
9,걸즈 앤 판처 최종장 제1화,애니메이션|드라마,2020,1.08,8.67,42,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|이구치 유카|오자키 마미|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191231_269/1...


In [37]:
df.shape

(12330, 11)

In [5]:
c = df['rating'].mean()
m = df['vote_count'].quantile(.6)
print(round(c,3), round(m,3))

def weighted_vote_average(record):
    v, r = record['vote_count'], record['rating']
    return (v/(v+m))*r + (m/(m+v))*c

df['weighted_vote'] = df.apply(weighted_vote_average,axis=1)

7.187 173.0


### 제목 검색

In [6]:
search = '어벤 져스'
df[df['title'].apply(lambda x: x.replace(' ','')).str.startswith(search.replace(' ',''))].sort_values('weighted_vote',ascending=False)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,weighted_vote
430,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68905,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20190417_250/1...,9.374507
1219,어벤져스: 인피니티 워,액션|모험|판타지|SF,2018,4.25,8.96,35881,"새로운 조합을 이룬 어벤져스,\r 역대 최강 빌런 타노스에 맞서 세계의 운명이 걸린...",로버트 다우니 주니어|조슈 브롤린|크리스 헴스워스|베네딕트 컴버배치|크리스 프랫|마...,세바스찬 스탠|기네스 팰트로|폼 클레멘티에프|레티티아 라이트|카렌 길런|다나이 구리...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20180322_242/1...,8.951490
6423,어벤져스,액션|SF|모험,2012,4.26,8.80,17954,"지구의 안보가 위협당하는 위기의 상황에서 슈퍼히어로들을 불러모아 세상을 구하는, 일...",로버트 다우니 주니어|스칼렛 요한슨|크리스 헴스워스|크리스 에반스|마크 러팔로|제레...,기네스 팰트로|코비 스멀더스|폴 베타니|클락 그레그|루 페리그노|스텔란 스카스가드|...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20120426_172/1...,8.784602
3847,어벤져스: 에이지 오브 울트론,액션|모험|판타지|SF,2015,4.23,8.31,31268,"쉴드의 숙적 히드라는 연구를 통해 새로운 능력자 막시모프 남매를 탄생시키고, 히드라...",로버트 다우니 주니어|크리스 헴스워스|마크 러팔로|크리스 에반스|스칼렛 요한슨|제레...,애런 존슨|엘리자베스 올슨|폴 베타니|코비 스멀더스|스텔란 스카스가드|수현|토마스 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20150324_33/14...,8.303818
1218,어벤져스 그림: 시간 전쟁,액션|모험,2018,10.30,5.30,10,시공간의 균열로 인해 마법 세계와의 통로가 열렸다! 아틀란티스의 여왕 막다는 마법의...,로렌 파킨슨,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20181102_272/1...,7.083456
5724,어벤져스 어셈블,애니메이션,2013,5.26,5.40,15,"세계 최강의 수퍼 히어로가 뭉쳤다. 아이언맨, 헐크, 캡틴 아메리카, 호크아이, 블...",NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20150918_131/1...,7.044003
429,어벤져스 오브 저스티스,액션|코미디|가족,2019,10.16,2.80,10,은하계 최강의 빌런 조크스터는 태양을 없애 지구에 새로운 빙하기를 불러일으키고자 한...,에이미 스마트|토니 차발레로|사이먼 렉스|스티븐 랜나지시|제프 체이스|제이슨 앨런 스미스,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191022_233/1...,6.946844
